In [27]:
import requests
from bs4 import BeautifulSoup
import re
import time
import numpy as np

In [2]:
def get_style_links():
    # we start on the styles page because it is the easiest page 
    # from which to access all beers
    
    followable_links = []
    url = 'https://www.beeradvocate.com/beer/styles/'
    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page, "lxml")
    
    # get style links

    styles_links = [link for link in soup.find_all('a') if 'beer/styles' in str(link)]

    styles_nums = []

    for link in styles_links[2:]:
        styles_nums.append(str(link).split('/')[3])
        
    styles_nums = sorted(styles_nums)[1:]
    
    for num in styles_nums:
        followable_links.append(url + num)
    
    return followable_links

In [3]:
style_links = get_style_links()
style_links[0]

'https://www.beeradvocate.com/beer/styles/10'

In [4]:
def get_brew_beer_links(url):
    
    followable_links = []
    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page, "lxml")
    
    beer_links = [link for link in soup.find_all('a') if '/beer/profile/' in str(link)]
    
    beer_nums = []

    for link in beer_links:
        beer_num = str(link).split('/')
        if len(beer_num) > 6:

            # take both brewering number and beer number

            brew, beer = beer_num[3:5]
            brew_beer = brew + '/' + beer + '/'
            beer_nums.append(brew_beer)
        else:
            pass
    
    prof_url = 'https://www.beeradvocate.com/beer/profile/'
    
    for num in beer_nums:
        followable_links.append(prof_url + num)
    
    return followable_links
    

In [5]:
brew_beer_link = get_brew_beer_links(style_links[0])
brew_beer_link[0]

'https://www.beeradvocate.com/beer/profile/388/5281/'

Do I also want to separately track score within beer type??? I don't think I've gotten that...

In [29]:
def get_beer_info(url):
    
    # Getting soup
    
    key = []
    values = []
    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page, "lxml")
    
    # Finding feature values
    
    title_div = soup.find(class_='titleBar')
    beer_brew_names = str(title_div.text).replace('\n','')
    
    score_obj = soup.find('span', {'class': 'BAscore_big'})
    score = float(re.split(r'>|<', str(score_obj))[4])
    
    beer_class_obj = soup.find_all('b')[4]
    beer_class = re.split(r'>|<', str(beer_class_obj))[2]
    
    stats_objs = soup.find_all('dd')[5:13]
    stats_list = []
    for item in stats_objs:
        item = str(item.text).strip()
        if len(item) > 0:
            clean_item = re.sub(r'#|%|,','',item)
            if '.' in clean_item:
                stats_list.append(float(clean_item))
            else:
                stats_list.append(int(clean_item))
        else:
            pass
    ranking, reviews, ratings, pDev, wants, gots, trade = stats_list
    
    brew_objs = soup.find_all('a')[119:123]
    brew_objs_list = []
    for obj in brew_objs:
        brew_objs_list.append(obj.text)
    brew, region, site, style = brew_objs_list
    
    abv_div = soup.find('div',attrs={'id':'info_box'})
    abv = str(abv_div).split('(ABV):</b>')[1][1:5]
    
    availability_str = str(abv_div).split('Availability:</b> ')[1]
    availability = availability_str.split('\n')[0]
    
    comm_desc_str = str(abv_div).split('Description:</b>\n<br/>\n')[1]
    comm_desc = comm_desc_str.split('<br')[0]
    
    date_added_str = str(abv_div).split('<br/><br/>')[-2]
    date_added = date_added_str.split()[-1]
    
    key = beer_brew_names 
    
    values = [score, beer_class, ranking, reviews, ratings, pDev, 
                            wants, gots, trade, brew, region, site, style, abv, 
                            availability, comm_desc, date_added]
    
    return key, values

In [11]:
key, values = get_beer_info(brew_beer_link[0])
print(key)
print(values)
dic = {}
dic[key] = dic[values]

Cantillon Fou' Foune | Brasserie Cantillon
[4.65, 'World-Class', 23, 562, 3079, 7.31, 2811, 669, 58, 'Brasserie Cantillon', 'Belgium', 'cantillon.be', 'Belgian Fruit Lambic', 5.0, 'Rotating', 'Apricot Lambic', '08-05-2002']


TypeError: unhashable type: 'list'

In [16]:
dic = {}

In [17]:
score, beer_class, ranking, reviews, ratings, pDev, \
wants, gots, trade, brew, region, site, style, abv, \
availability, comm_desc, date_added = values

## Master Function

In [ ]:
def get_beer_info():
    
    style_links = get_style_links()
    
    for style_link in style_links[0]:
        
        
        
        brew_beer_links = get_brew_beer_links(style_link)
        
        for brew_beer_link in brew_beer_links[0]:
            
            get_beer_info(brew_beer_link)
    
    
    
    

In [ ]:
get_beer_info()

In [25]:
def compile_beer_info():
    
    dic = {}
    
    style_links = get_style_links()

    brew_beer_links = get_brew_beer_links(style_links[0])

    for brew_beer_link in brew_beer_links:
        
        key, values = get_beer_info(brew_beer_link)
        
        dic[key] = values
        
        time.sleep(np.random.poisson(10)/10)
        
        #Write to csv
        
        
    
    return dic

In [30]:
%%time
test = compile_beer_info()

CPU times: user 3.78 s, sys: 87.9 ms, total: 3.87 s
Wall time: 1min 10s


In [35]:
len(list(test.keys()))

50

In [36]:
test

{"Cantillon Fou' Foune | Brasserie Cantillon": [4.65,
  'World-Class',
  23,
  562,
  3079,
  7.31,
  2811,
  669,
  58,
  'Brasserie Cantillon',
  'Belgium',
  'cantillon.be',
  'Belgian Fruit Lambic',
  '5.00',
  'Rotating',
  'Apricot Lambic',
  '08-05-2002'],
 'Lindemans Framboise | Brouwerij Lindemans': [3.88,
  'Very Good',
  15295,
  1065,
  2884,
  14.43,
  81,
  437,
  4,
  'Brouwerij Lindemans',
  'Belgium',
  'lindemans.be',
  'Belgian Fruit Lambic',
  '2.50',
  'Year-round',
  '2.5-4.0% ABV',
  '11-01-2000'],
 'Cantillon Kriek 100% Lambic | Brasserie Cantillon': [4.36,
  'Outstanding',
  569,
  737,
  2512,
  9.63,
  668,
  625,
  64,
  'Brasserie Cantillon',
  'Belgium',
  'cantillon.be',
  'Belgian Fruit Lambic',
  '5.00',
  'Year-round',
  'No notes at this time.',
  '10-06-2001'],
 'Cantillon Rosé De Gambrinus | Brasserie Cantillon': [4.32,
  'Outstanding',
  830,
  773,
  2444,
  9.49,
  515,
  638,
  60,
  'Brasserie Cantillon',
  'Belgium',
  'cantillon.be',
  'Belgi